In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [2]:
import dipy.reconst.sfm as sfm
import dipy.reconst.csdeconv as csd
import dipy.reconst.dti as dti
import dipy.core.gradients as grad
import dipy.reconst.cross_validation as xval
import dipy.data as dpd

In [3]:
import utils
import model as mm 
from model import Model, BiExponentialIsotropicModel

In [4]:
from parallelization import parallelization

In [5]:
mm.my_responses

[[0.001, 0.0, 0.0],
 [0.001, 0.00050000000000000001, 0.00050000000000000001],
 [0.0015, 0.0, 0.0],
 [0.0015, 0.00050000000000000001, 0.00050000000000000001],
 [0.0015, 0.001, 0.001]]

In [6]:
data = utils.read_data()

In [7]:
data1 = data['seen']['signal'][:, 0][::2]
bvals1 = data['seen']['bvals'][::2]
bvecs1 = data['seen']['bvecs'][::2]
delta1 = data['seen']['delta'][::2]
Delta1 = data['seen']['Delta'][::2]
te1 = data['seen']['TE'][::2]
g1 = data['seen']['g'][::2]

data2 = data['seen']['signal'][:, 0][1::2]
bvals2 = data['seen']['bvals'][1::2]
bvecs2 = data['seen']['bvecs'][1::2]
delta2 = data['seen']['delta'][1::2]
Delta2 = data['seen']['Delta'][1::2]
te2 = data['seen']['TE'][1::2]
g2 = data['seen']['g'][1::2]

In [8]:
gtab1 = grad.gradient_table(bvals1, bvecs1, big_delta=Delta1, small_delta=delta1)
gtab2 = grad.gradient_table(bvals2, bvecs2, big_delta=Delta2, small_delta=delta2)

In [9]:
sphere = dpd.get_sphere()

In [10]:
sphere = sphere.subdivide(1)

In [11]:
sphere.x.shape

(1442,)

In [12]:
p = parallelization()

In [13]:
models = p.start(Model, 2, [gtab1, gtab2], sphere=[sphere], isotropic=[BiExponentialIsotropicModel])

Parallelization starts on 4 CPUs.
100%      0.00min remaining
Total Time needed: 0.00min
 50%      0.01min remaining


In [14]:
fits = p.start([i.fit for i in models], 2, [data1, data2], [te1, te2])

Parallelization starts on 4 CPUs.
100%      0.00min remaining
Total Time needed: 1.79min
 50%      0.00min remaining


In [15]:
predicts = p.start([i.predict for i in fits] + [i.predict for i in fits], 4, [gtab2, gtab1, gtab1, gtab2], [te2, te1, te1, te2])

Parallelization starts on 4 CPUs.
100%      0.00min remaining
Total Time needed: 2.82min
 25%      0.01min remaining
 50%      0.01min remaining
 75%      0.00min remaining


In [16]:
print("Fit 1 => Predict 2: LSE=%s"%utils.LSE(predicts[0], data2))
print("Fit 2 => Predict 1: LSE=%s"%utils.LSE(predicts[1], data1))
print("Fit 1 => Predict 1: LSE=%s"%utils.LSE(predicts[2], data1))
print("Fit 2 => Predict 2: LSE=%s"%utils.LSE(predicts[3], data2))
np.mean([utils.LSE(predicts[0], data2), utils.LSE(predicts[1], data1)])

Fit 1 => Predict 2: LSE=2.72744449274
Fit 2 => Predict 1: LSE=2.67612369779
Fit 1 => Predict 1: LSE=2.65999732383
Fit 2 => Predict 2: LSE=2.73660123248


2.7017840952674641